In [1]:
import altair as alt
alt.data_transformers.enable('json')
import pandas as pd
babyNames = pd.read_csv("dpt2020.csv", sep=";")
babyNames.head()

,sexe,preusuel,annais,dpt,nombre
0,1,_PRENOMS_RARES,1900,02,7
1,1,_PRENOMS_RARES,1900,04,9
2,1,_PRENOMS_RARES,1900,05,8
3,1,_PRENOMS_RARES,1900,06,23
4,1,_PRENOMS_RARES,1900,07,9


In [4]:

noDtpNames = babyNames.groupby(['sexe', 'preusuel', 'annais'], as_index=False).agg({'nombre': 'sum'})
most_common_names = noDtpNames.loc[noDtpNames.groupby(['sexe', 'annais'])['nombre'].idxmax()]

# percentage 
total_by_year = noDtpNames.groupby(['sexe', 'annais'])['nombre'].sum().reset_index()
most_common_names = most_common_names.merge(total_by_year, on=['sexe', 'annais'], suffixes=('', '_total'))
most_common_names['pourcentage'] = most_common_names['nombre'] / most_common_names['nombre_total'] * 100

#male_names = most_common_names[most_common_names['sexe'] == 1]

#male
male = alt.Chart(most_common_names).mark_bar().encode(
    x=alt.X('pourcentage:Q', title='Proportion nom masculin (%)', scale=alt.Scale(reverse=True)),
    y=alt.Y('annais:O', title='Année', sort=alt.EncodingSortField(field='annais', order='descending'), axis=None)
).transform_filter(
    alt.datum.sexe == 1
).properties(height=alt.Step(20))

textmale = male.mark_text(
    align='right',
    baseline='middle',
    dx=-3  
).encode(
    text='preusuel:N'
)
male_chart = male + textmale

#female
female = alt.Chart(most_common_names).mark_bar().encode(
    x=alt.X('pourcentage:Q', title='Proportion nom feminin (%)'),
    y=alt.Y('annais:O', title='Année', sort=alt.EncodingSortField(field='annais', order='descending'))
).transform_filter(
    alt.datum.sexe == 2
).properties(height=alt.Step(20))

textfemale = female.mark_text(
    align='left',
    baseline='middle',
    dx=3  
).encode(
    text='preusuel:N'
)
female_chart = female + textfemale


# showing graph
combined_chart = alt.hconcat(male_chart, female_chart).resolve_scale(y='shared')

combined_chart

alt.HConcatChart(...)